In [1]:
from data import load_data, GS_Dataset, make_gs_dataset, GS_Results
from model import Narx, DEVICE
from util_fun import train_narx_simval, narx_sim_nrms
import torch
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split



In [2]:
n_a=15
n_b=15
n_layers=5
n_nodes=50
n_epochs=15000

In [3]:
model=Narx(n_a+n_b,n_nodes, n_layers).to(DEVICE)

In [4]:
x_data, y_data= load_data()
data = make_gs_dataset(x_data, y_data, n_a, n_b, DEVICE)

In [5]:
results:GS_Results = train_narx_simval(model, n_a, n_b, data, n_epochs=n_epochs)

Checkpoint at epoch 1:  

current best pred NRMS: 1.0003511707381672, previous best pred NRMS: inf 

current best sim NRMS: 1.0019906511742223, previous best sim NRMS: inf 

Checkpoint at epoch 601:  

current best pred NRMS: 0.46439638982082637, previous best pred NRMS: 1.0003511707381672 

current sim NRMS: 1.2622884985393292, current best sim NRMS: 1.0019906511742223 

Checkpoint at epoch 1201:  

current best pred NRMS: 0.4184390616484508, previous best pred NRMS: 0.46439638982082637 

current sim NRMS: 1.3305224133542497, current best sim NRMS: 1.0019906511742223 

Checkpoint at epoch 1801:  

current best pred NRMS: 0.37421825243091156, previous best pred NRMS: 0.4184390616484508 

current sim NRMS: 1.5497360682437114, current best sim NRMS: 1.0019906511742223 

Checkpoint at epoch 2401:  

current best pred NRMS: 0.3606928030402896, previous best pred NRMS: 0.37421825243091156 

current sim NRMS: 1.6747513743781939, current best sim NRMS: 1.0019906511742223 

Checkpoint at epoch

In [6]:
torch.save(results.best_model.state_dict(), 'narx15K_na15_nb15_nlay5_nnode50_pred')